# Домашнаяя работа №2. 
# Автор: Серегин М. С

In [1]:
import os
import numpy as np
from itertools import product
import pandas as pd
import numpy as np
import random
from surprise import Dataset
from surprise import Reader
import tqdm
import matplotlib.pyplot as plt
import tqdm

# Метрики

In [2]:
def MAP(ranked_arr, true_rel, n):
    """
    На вход подается отранжированный массив индексов и массив релевантности каждого элемента по порядку. 
    Example of input:
        ranked_arr = [[2,3,1,0],
                      [1,0,2,4]]
                      
        true_rel   = [[0,0,1,0,1,1,1],
                        [0,0,1,1,1,1]]
    """
    
    def precision_k(ranked_arr, true_rel, k):

        ind = ranked_arr[:,:k]
        rels = np.take_along_axis(true_rel, ind, axis=1) != 0 
        return np.mean(rels, axis=1)
    
    def avg_precision_n(ranked_arr, true_rel, n):

        sum_precision_n = np.zeros(ranked_arr.shape[0])
        m = min(n, ranked_arr.shape[1])
        for k in range(1,m+1):
            arr_precision = precision_k(ranked_arr, true_rel, k)
            ind = ranked_arr[:,k-1].reshape(-1,1)  
            rel_exact_k = np.take_along_axis(true_rel, ind, axis=1).reshape(-1) != 0
            out = rel_exact_k * arr_precision
            sum_precision_n += out

        return sum_precision_n / m
        
        
    return np.mean(avg_precision_n(ranked_arr, true_rel, n))
        
        
    
        
        

In [3]:
def test_MAP():
    
    ranked_arr = np.array([[2,3,1,0],
                          [1,0,2,4]])
    true_rel   = np.array([[0,0,1,0,1,1],
                    [0,0,1,1,1,1]])
    
    assert np.abs(MAP(ranked_arr, true_rel, 1) - 0.5) < 0.001
    assert np.abs(MAP(ranked_arr, true_rel, 2) - 0.25) < 0.001
    assert np.abs(MAP(ranked_arr, true_rel, 3) - 0.222) < 0.001
    assert np.abs(MAP(ranked_arr, true_rel, 4) - 0.229) < 0.001
    
test_MAP()

In [4]:
def MRR(ranked_arr, true_rel):
    """
    На вход подается отранжированный массив индексов и массив релевантности каждого элемента по порядку. 
    Example of input:
    ranked_arr = np.array([[2,3,1,0],
              [1,0,2,4]])
    true_rel   = np.array([[0,0,1,0,1,1],
                    [0,0,1,1,1,1]])
    """
        
    zero = np.zeros(ranked_arr.shape[0]) -1
    ax_0, ax_1 = np.where(np.take_along_axis(true_rel, ranked_arr, axis=1) !=0) # релевантный , если != 0

    for par in zip(ax_0, ax_1):
        if zero[par[0]] == -1:
            zero[par[0]] = par[1] + 1
            

            
    return  np.sum(1 / zero[zero > 0]) / ranked_arr.shape[0]

    
    
    

In [5]:
def testMRR():
    ranked_arr = np.array([[2,3,1,0],
                            [1,0,2,4],
                            [2,1,0,3],
                            [2,3,0,1]])
    
    true_rel   = np.array([[0,0,1,0,1,1],
                            [0,0,1,1,1,1],
                          [0,1,0,1,1,1],
                          [0,0,0,0,0,0]])
    assert np.abs(MRR(ranked_arr, true_rel) - 0.458) < 0.001
testMRR()

In [6]:
def NDCG(ranked_arr, true_rel, n):
    """
    Расчитывает метрику как для одного вектора, так и для нескольких, объединенных в массив. 
    В любом случае, размерность входных массивов = 2. 
    """
    
    def DCG(ranked_arr, true_rel, n):
        ind = 2**np.take_along_axis(true_rel, ranked_arr[:,:n], axis=1) - 1
        log = np.log(np.arange(2,ind.shape[1]+2)) 
      
        res = ind / log
        return np.sum(res,axis=1)
    
    def IDCG(ranked_arr, true_rel, n):
        k = min(ranked_arr.shape[1],n)
        return np.sum(
            (2**np.sort(true_rel,axis=1)[:,-1:-k-1:-1]-1) / np.log(np.arange(2,k+2)),
            axis=1
        )
        
    dcg = DCG(ranked_arr, true_rel, n)
    idcg = IDCG(ranked_arr, true_rel, n)

    return dcg / idcg
    
    

In [15]:
# тестирование
r = np.array([[3,2,3,0,1,2]])
o = np.array([[0,1,2,3,4,5]])
assert (NDCG(o,r,5) - np.array([0.875,])).sum() < 0.01

# LambdaRank 

In [8]:
class LambdaRank:
    def __init__(self,lr=0.1, sigma=1, num_epochs=3, seed=1, metric=NDCG,reg=0.05):

        self.reg=reg
        self.lr = lr
        self.sigma = sigma
        self.reg = 0.05
        self.num_epochs = num_epochs
        self.seed = seed
        
    def fit(self,df):
        """
        На вход принимает pd.DataFrame, который имеет следующие столбцы: relevance, qid, *range(0,n). 
        n - количество признаков
        """

        n = len(df.columns) - 2
        w = np.random.randn(1,n) 

        
        for epoch in range(self.num_epochs):
            mistakes = 0
            k = 0
            for qid in df.qid.unique():
                current_df = df[df.qid == qid].sort_values("relevance", ascending=False).copy()

                arr_relevance = np.array(current_df.relevance)
                F = current_df.drop(["relevance", "qid"],axis=1).to_numpy()

                arr_values = (w @ F.T).reshape(-1)
                
                zero = np.zeros(len(arr_values)).astype(np.int32)  
                for pos, i in enumerate(np.argsort(-arr_values)):
                    zero[i] = pos

                # берем случайную пару и делаем шаг
                
                for i in range(len(current_df)): # n
                    n_1, n_2 = np.sort(np.random.choice(range(len(current_df)), 2, replace=False)) # объекты в датасете отсортированы!
                    
              
                    
                    x1, x2 = F[n_1], F[n_2]
                    r1, r2 = int(arr_relevance[n_1]), int(arr_relevance[n_2])
                    o1,o2  = int(zero[n_1]), int(zero[n_2])
                    

                    assert (type(r1) == int) and (type(o1) == int) 
                    assert (type(r2) == int) and (type(o2) == int) 

                    
                    o1 += 1
                    o2 += 1 # так как при сортировке начинается с 0
                    
                    res1 = float(w@(x1-x2)) 
                    if r1 > r2 and res1 < 0:
                        mistakes += 1

                    if r1 > r2:
                        k+=1
                        delta_NDCG = np.abs((r1 / np.log(1 + o1) + r2 / np.log(1 + o2)) - (r1 / np.log(1 + o2) + r2 / np.log(1 + o1)))
                        w = w + self.lr * self.sigma / (1 + np.exp(self.sigma * float(w@(x1-x2)))) * delta_NDCG * (x1 - x2).reshape(w.shape) 

                        assert res1 <= float(w@(x1-x2))



        # trainig is completed   
        self.w = w
        
        
    def compute_metric_for_all_queries(self,df_val, n,metric=NDCG):
        lst_metric = []
        for qid_ in df_val.qid.unique():
            current_df = df_val[df_val.qid == qid_]
            
            arr_relevance = np.array(current_df.relevance).reshape(1,-1)
            F = current_df.drop(["relevance", "qid"],axis=1).to_numpy()
            
            order = np.argsort(-(self.w @ F.T).reshape(1,-1))


            if (arr_relevance!=0).sum()>1: # исключаем те порядки, где нет ни одного релевантного - любой порядок будет эквивалентен

                exact_metric = metric(order, arr_relevance,n) if metric is NDCG or metric is MAP else (
                                                                                metric(order, arr_relevance))

                lst_metric.append(float(exact_metric))

        return np.array(lst_metric)
        
        
    

# Test LambdaRank on MQ2007, MQ2008

In [9]:
train_dataset_MQ2007 = pd.DataFrame([])
for s in ["S1.txt","S2.txt","S3.txt","S4.txt"]:
    df = pd.read_csv(s, sep=" ",names=range(57)).drop(labels=list(range(48,57)),axis=1)
    df = df.replace(r'^.+:', '', regex=True)
    df.columns = ["relevance", "qid",*list(range(46))]
    df[["relevance", "qid"]] = df[["relevance", "qid"]].astype(np.int32)
    df[[*list(range(46))]] = df[[*list(range(46))]].astype(np.float32)
    
    train_dataset_MQ2007 = pd.concat((train_dataset_MQ2007,df))
    

# val_dataset
for s in ["S5.txt",]:
    df = pd.read_csv(s, sep=" ",names=range(57)).drop(labels=list(range(48,57)),axis=1)
    df = df.replace(r'^.+:', '', regex=True)
    df.columns = ["relevance", "qid",*list(range(46))]
    df[["relevance", "qid"]] = df[["relevance", "qid"]].astype(np.int32)
    df[[*list(range(46))]] = df[[*list(range(46))]].astype(np.float32)
    
    val_datasetMQ2007 = df.copy()

In [10]:
train_dataset_MQ2008 = pd.DataFrame([])
for s in ["S1.txt","S2.txt","S3.txt","S4.txt"]:
    df = pd.read_csv("MQ2008/" +s , sep=" ",names=range(57)).drop(labels=list(range(48,57)),axis=1)
    df = df.replace(r'^.+:', '', regex=True)
    df.columns = ["relevance", "qid",*list(range(46))]
    df[["relevance", "qid"]] = df[["relevance", "qid"]].astype(np.int32)
    df[[*list(range(46))]] = df[[*list(range(46))]].astype(np.float32)
    
    train_dataset_MQ2008 = pd.concat((train_dataset_MQ2008,df))
    

# val_dataset
for s in ["S5.txt",]:
    df = pd.read_csv("MQ2008/" +  s, sep=" ",names=range(57)).drop(labels=list(range(48,57)),axis=1)
    df = df.replace(r'^.+:', '', regex=True)
    df.columns = ["relevance", "qid",*list(range(46))]
    df[["relevance", "qid"]] = df[["relevance", "qid"]].astype(np.int32)
    df[[*list(range(46))]] = df[[*list(range(46))]].astype(np.float32)
    
    val_datasetMQ2008 = df.copy()

#### MQ2008

In [1359]:
Lambdarank = LambdaRank(lr=0.01,
    sigma=1,
    num_epochs=10,
    seed=1,
    reg=0)
Lambdarank2.fit(train_dataset_MQ2008)

In [1360]:
#NDCG
for n in [3,5,10,15]:
    res = Lambdarank.compute_metric_for_all_queries(val_datasetMQ2008,n,NDCG)
    print(f"Средний NDCG@{n}", np.mean(res))

Средний NDCG@3 0.6495358772970298
Средний NDCG@5 0.7120044380013448
Средний NDCG@10 0.7707059361165118
Средний NDCG@15 0.7815778766635076


In [1362]:
#MRR
res = Lambdarank.compute_metric_for_all_queries(val_datasetMQ2008,n,MRR)
print(f"Средний MRR", np.mean(res))

Средний MRR 0.8695512820512821


In [1364]:
# MAP
for n in [1,2,3,4]:
    res = Lambdarank.compute_metric_for_all_queries(val_datasetMQ2007,n,MAP)
    print(f"Средний MAP@{n}", np.mean(res))

Средний MAP@1 0.5945945945945946
Средний MAP@2 0.5463320463320464
Средний MAP@3 0.5053625053625053
Средний MAP@4 0.4873712998712999


#### MQ2007

In [1366]:
Lambdarank = LambdaRank(lr=0.01,
    sigma=1,
    num_epochs=10,
    seed=10,
    reg=0)

Lambdarank2.fit(train_dataset_MQ2007)

In [1250]:
#NDCG
for n in [3,5,10,15]:
    res = Lambdarank.compute_metric_for_all_queries(val_datasetMQ2007,n,NDCG)
    print(f"Средний NDCG@{n}", np.mean(res))

Средний NDCG@1 0.5546975546975548
Средний NDCG@3 0.5332435869100559
Средний NDCG@5 0.5324426825383389
Средний NDCG@10 0.5586352284219644
Средний NDCG@15 0.5925793844450021


In [1367]:
#MRR
res = Lambdarank.compute_metric_for_all_queries(val_datasetMQ2007,10,MRR)
print(f"Средний MRR", np.mean(res))

Средний MRR 0.7174852457415483


In [1327]:
# MAP
for n in [1,2,3,4]:
    res = Lambdarank.compute_metric_for_all_queries(val_datasetMQ2007,n,MAP)
    print(f"Средний MAP@{n}", np.mean(res))

Средний MAP@1 0.637065637065637
Средний MAP@2 0.5801158301158301
Средний MAP@3 0.5373230373230374
Средний MAP@4 0.5043436293436293


# Подготовка обучающего и валидационного датасета 
# (датасет Movielens)

#### В данном блоке мы сконструируем два датасета. 
#### Из тренировочного датасета будут исключены все тройки (user,item,rating), которые используются в валидации. 
#### Каждому пользователю, купившему хотя бы 70 товаров сопоставляется валидационный датасет, который необходимо отранжировать. На этих данных будет замеряться качество модели. 

## ? Как адаптировать датасет для ранжирования к задаче рекомендаций?

Для каждого запроса нам дана релевантность. 
Можно составить матрицу R, где по строчкам будут отражены конкретные запросы, по столбцам все возможные документы. На пересечении столбца j и строчки i будет стоять релевантность документа j запросу i , если имеется и NAN в противном случае. 
Данная матрица может использоваться в качестве обучения. 

В исходном датасете имеются оценки 5,4,...1. 
Переведем их в соответствующие оценки, отражающие релевантность предмета. \
5 -> 1 \
4 -> 0.5 \
3 -> 0\
2 -> 0 \
1 -> 0 

In [1114]:
def pivot_to_df(pivot_, user, item, rating):
    """
    Функция по поданной сводной таблице строит pandas.DataFrame. 
    Вход: сводная таблица, например 
    >>> data = Dataset.load_builtin('ml-100k')
    Выход: pandas.DataFrame
    """
    pivot = pivot_.copy()
    pivot[user] = pivot[user].astype(int)
    pivot[item] = pivot[item].astype(int)
    pivot[rating] = pivot[rating].astype(int)

    new_df = pd.DataFrame(index=range(1, max(pivot[user])+1), columns=range(1, max(pivot[item])+1))

    for i in pivot.index:
        curuser, curitem, currating = pivot.loc[i]

        new_df.loc[curuser,curitem] = currating

        
    return new_df 


def compute_ml():
    dataml = Dataset.load_builtin("ml-100k")
    ml = pd.DataFrame(dataml.raw_ratings)
    ml.columns = ['user', 'item', 'rating', 'timestamp']
    ml.user = ml.user.astype(np.int32)
    ml.item = ml.item.astype(np.int32)
    ml.rating = ml.rating.astype(np.int32)
    del ml["timestamp"]
    return ml

In [1115]:
ml =compute_ml()

In [1116]:
freq_u = pd.DataFrame(ml.groupby(by="user")["user"].count()) # находим тех пользователей, которые купили, как минимум, 70 предметов.

idx = np.array(freq_u["user"] > 70) # 446

In [1117]:
users_in_val = np.array(freq_u[idx].index) # 446 * len of ranking list=15 => валидационная выборка равна 6690

In [1118]:
# для каждого пользователя из users_in_val добавим случайные 15 предметов в валидационный датасет. 

In [1119]:
dct_val = {} # В данном словаре находятся валидационные датасеты для каждого пользователя, купившего хотя бы 70 товаров. 
np.random.seed(1)
for u in users_in_val:
    sample = ml[ml["user"] == u].copy()

    val = sample.sample(n=15).copy()


    
    
    val.rating[val.rating < 4] = 0 
    val.rating[val.rating == 5] = 1
    val.rating[val.rating == 4] = 0.5

    
    while (np.array(val.rating) == 0).sum() > 13: # если в выборку попали более 12 нерелевантных значений
        val = sample.sample(n=15)
        
        val.rating[val.rating < 4] = 0 
        val.rating[val.rating == 5] = 1
        val.rating[val.rating == 4] = 0.5
        
        print("У этого пользователя мало релевантных объектов", u)
        
        
    ml = ml.drop(np.array(val.index), axis=0)
    
    dct_val[u] = val
    
    

У этого пользователя много нерелевантных объектов 102
У этого пользователя много нерелевантных объектов 181
У этого пользователя много нерелевантных объектов 181
У этого пользователя много нерелевантных объектов 454
У этого пользователя много нерелевантных объектов 637


In [1120]:
df_for_ranksvd = pivot_to_df(ml, "user", "item", "rating") # df for RankSVD

# LambdaRank_movie



### Алгоритм немного модифицируем. 
### Теперь в качестве признаков используем признаки из разложения SVD.

In [1255]:
class LambdaRank_movie:
    def __init__(self,lr=0.1, sigma=1, num_epochs=3, seed=1, metric=NDCG,reg=0.05):
        self.lr = lr
        self.sigma = sigma
        self.reg = 0.05
        self.num_epochs = num_epochs
        self.seed = seed
        self.reg=reg
        
    def _create_features(self, th_user, th_item):
        """
        Gets value of user, value of item. Returns vector of features of item th_item for query (in our case user) th_user
        """

        pq = self.P[th_user-1,:] * self.Q[:,th_item-1] # в матрице индекс на 1 меньше
        p_u = np.array([self.b_u[th_user-1]])
        p_i = np.array([self.b_i[th_item-1]])
        p_mu = np.array([self.mu])

        x_features = np.concatenate((p_mu,p_u, p_i, pq)).reshape(-1,1)
        
        return x_features
        
    def fit(self,P_input, Q_input, b_u,b_i,mu, ml):
        """
        На вход принимает четыре матрицы: P, Q, b_u, b_i, полученные факторизацией R.
        Для каждого запроса (в нашем случае для каждого пользователя)
            и для каждого предемета считается вектор признаков следующим образом:
            
        features(u,i) = (mu,b_u, b_i, P_u1*Q_i1, ...,P_uf*Q_if).
        Помимо этого, принимает сводную таблицу ml со столбцами user, item, rating.
        Модель обучается стохастическим градиентым спуском. 
        """

        self.P = P_input
        self.Q = Q_input
        self.b_u = b_u
        self.b_i = b_i
        self.mu = mu
        
        
        dct_ml = {} 
        dct_size_of_user = {}
        total_size = 0 # подсчет количества оцененных объектов

        for u in ml.user.unique():
            dct_ml[u] = ml[ml["user"] == u].sort_values("rating", ascending=False) # нужно ли копировать? чтобы случайно не изменить ml?
            
            dct_size_of_user[u] = len(dct_ml[u])
            total_size += len(dct_ml[u])

            
        n_factors = self.P.shape[1]
            
        w = np.random.randn(1,n_factors + 1 + 1 + 1) # P_u * Q_i + b_u + b_i + mu
        
        for epoch in range(self.num_epochs):

            for user in tqdm.tqdm(ml.user.unique()):
                
            
                current_df = dct_ml[user].copy()
                
                # расчитываем a(x,w) для каждой пары (u,i) в current_df -> array
                lst_values = []
                for j in range(len(current_df)):
                    row = current_df.iloc[j]
         
                    features = self._create_features(int(row["user"]), int(row["item"])) # почему-то берется float
                    
                    lst_values.append(float(w@features))
                    
                arr_values = np.array(lst_values)
                zero = np.zeros(len(arr_values))
                for pos, i in enumerate(np.argsort(-arr_values)): # почему во возрастанию???
                    zero[i] = pos

                current_df["my_order"] = zero # единицу не прибавил
                
                
                # берем случайную пару и делаем шаг
                
                for i in range(len(current_df)): # n
   
                    n_1, n_2 = np.sort(np.random.choice(range(len(current_df)), 2, replace=False)) # объекты в датасете отсортированы!
                    

                    a_1 = current_df.iloc[n_1]
                    a_2 = current_df.iloc[n_2]
        
                    u1, i1, r1, o1 = [int(a_1[i]) if i!=2 else float(a_1[i]) for i in range(len(a_1))]
                    u2, i2, r2, o2 = [int(a_2[i]) if i!=2 else float(a_2[i]) for i in range(len(a_2))]

                    assert (type(u1) == int) and (type(i1) == int) and (type(r1) == float) and (type(o1) == int) 
                    assert (type(u2) == int) and (type(i2) == int) and (type(r2) == float) and (type(o2) == int) 
                    
                    o1 += 1
                    o2 += 1 # так как при сортировке начинается с 0
                    
                    x1 = self._create_features(u1, i1)
                    x2 = self._create_features(u2, i2)     

                    if r1 > r2 or r2> r1:
                        delta_NDCG = np.abs((r1 / np.log(1 + o1) + r2 / np.log(1 + o2)) - (r1 / np.log(1 + o2) + r2 / np.log(1 + o1)))
                        w = w + self.lr * self.sigma / (1 + np.exp(self.sigma * float(w@(x1-x2)))) * delta_NDCG * (x1 - x2).reshape(w.shape) - self.reg*w

                
        # trainig is completed
        
        self.w = w
        
        
    def predict_one_order(self,pivot_predict):
        """
        На вход ожидается сводная таблица валидации для одного пользователя.
        """
        pivot = pivot_predict.copy()
        pred = np.zeros(shape=len(pivot.index))
        for i in range(len(pivot.index)):
            user, item = list(map(int, list(pivot.iloc[i])))
            

            
            features = self._create_features(user,item)
            pred[i] = float(self.w @ features)
            
   
        return np.argsort(pred).reshape(1,-1)[:,::-1] # возвращаем отранжированные индексы
        
    def predict_order_all(self,dct_val, len_of_order):
        """
        The function gets dictionary in which keys are users and values corresponding pivot_table need to be ranked.
        """
        predicted_orders = np.zeros((1,len_of_order))
        true_ratings_arr = np.zeros((1,len_of_order))
        for key in dct_val:

            order_of_indices = (self.predict_one_order(dct_val[key].drop("rating",axis=1))).astype(np.int32) # getting the order
            true_ratings = np.array(dct_val[key]["rating"]).reshape(1,-1)
            
            predicted_orders = np.concatenate((predicted_orders, order_of_indices))
            true_ratings_arr = np.concatenate((true_ratings_arr, true_ratings))
            
        
        return predicted_orders[1:].astype(np.int32),true_ratings_arr[1:]
        
        
    def compute_metric_for_all_queries(self,dct_val, len_of_order, n, metric):
        """
        Function gets dictionary in which keys are users and values corresponding pivot_table need to be ranked 
            , length of order,  n in NDCG
        """
        
        predicted_orders, true_ratings_arr = self.predict_order_all(dct_val, len_of_order)
        comp_metric = metric(predicted_orders, true_ratings_arr,n) if metric is NDCG or metric is MAP else (
                                                                                metric(predicted_orders, true_ratings_arr))

        return comp_metric
        
        
    

# RankSVD

#### В этом блоке SVD, сконструированный в прошлом домашнем задании, будет модифицирован для решения задач ранжирования.
#### В последствии алгоритм будет использоваться для формирования признаков пары (пользователь, предмет)

In [1268]:
class RankSVD:
    def __init__(self, n_factors=5, n_epochs=10, lr=0.001,reg=0.02,random_state=1):
        self.n_epochs = n_epochs
        self.n_factors = n_factors
        self.reg = reg
        self.random_state = random_state
        self.lr=lr
    
        
    def compute_us_U(self,RR, P,Q, I,mu, mask, notna_1):
        us = (np.sum((RR - P@Q - I - mu)*mask,axis=1)/notna_1).reshape(-1,1) # уцмножаю на маску, чтобы усреднение делать только по известным меткам
        U = us*np.ones_like(RR)
        return us, U

    def compute_it_I(self,RR,P,Q,U,mu, mask, notna_0):
        it = np.sum((RR - P@Q - U - mu)*mask, axis=0) / notna_0
        I = it*np.ones_like(RR)
        return it, I
    

    def _als(self,df_train):

        n_factors = self.n_factors
        random.seed(3)
        np.random.seed(3)
        RR = df_train.fillna(0).to_numpy()

        P = np.random.random((RR.shape[0],n_factors))
        Q = np.random.random((n_factors,RR.shape[1]))
        mask = RR != 0 
        R_est = RR + (P@Q)*~mask
        not_null = mask.sum()
        us = np.random.random((RR.shape[0],1))
        it = np.random.random((1,RR.shape[1]))

        U = us*np.ones_like(RR) 

        I = it*np.ones_like(RR)
        

        notna_0 = mask.sum(axis=0) + 0.00001
        notna_1 = mask.sum(axis=1) + 0.00001
        mu = np.ones_like(RR) * RR.sum() / not_null

        for _ in range(self.n_epochs):
            us, U = self.compute_us_U(RR, P,Q, I,mu, mask, notna_1)
            it, I = self.compute_it_I(RR,P,Q,U,mu, mask, notna_0)

            R_est = RR + (P@Q+I+U+mu)*~mask
            P = (np.linalg.inv((Q@Q.T)) @ (Q@R_est.T - Q@(U.T+I.T + mu.T))).T

            R_est = RR + (P@Q+I+U+mu)*~mask   
            Q = (np.linalg.inv((P.T@P))@(P.T @ R_est - P.T@(U+I+mu)))
    
        self.mu = mu
        self.P = P
        self.Q = Q
        self.I = I
        self.U = U

    def fit(self,df_train_):
        np.random.seed(self.random_state)
        df_train = df_train_.copy()
        self._als(df_train)

    def predict_one_order(self,pivot_predict):
        """
        На вход ожидается сводная таблица валидации для одного пользователя.
        """
        pivot = pivot_predict.copy()
        pred = np.zeros(shape=len(pivot.index))
        restored_R = self.P @ self.Q + self.U + self.I + self.mu
        
        for i in range(len(pivot.index)):
            user, item = list(map(int, list(pivot.iloc[i])))
            
            user -= 1
            item -= 1
            
            pred[i] = restored_R[user,item]
            
   
        return np.argsort(pred).reshape(1,-1)[:,::-1] # возвращаем отранжированные индексы
    
    
    def predict_order_all(self, dct_val, len_of_order):
        """
        The function gets dictionary in which keys are users and values corresponding pivot_table need to be ranked.
        """
        predicted_orders = np.zeros((1,len_of_order))
        true_ratings_arr = np.zeros((1,len_of_order))
        for key in dct_val:

            order_of_indices = (self.predict_one_order(dct_val[key].drop("rating",axis=1))).astype(np.int32) # getting the order
            true_ratings = np.array(dct_val[key]["rating"]).reshape(1,-1)
            
            predicted_orders = np.concatenate((predicted_orders, order_of_indices))
            true_ratings_arr = np.concatenate((true_ratings_arr, true_ratings))
            
        
        return predicted_orders[1:].astype(np.int32),true_ratings_arr[1:]
    
    
    def compute_metric_for_all_queries(self,dct_val, len_of_order, n, metric):
        """
        Function gets dictionary in which keys are users and values corresponding pivot_table need to be ranked 
            , length of order,  n in NDCG
        """
        
        predicted_orders, true_ratings_arr = self.predict_order_all(dct_val, len_of_order)
        comp_metric = metric(predicted_orders, true_ratings_arr,n) if metric is NDCG or metric is MAP else (
                                                                                metric(predicted_orders, true_ratings_arr))

        return comp_metric
   

In [1284]:
svd = RankSVD(n_factors=1)
svd.fit(df_for_ranksvd)

# Testing LambdaRank_movie on Movielens dataset

В исходном датасете имеются оценки 5,4,...1. 
Переведем их в соответствующие оценки, отражающие релевантность предмета. \
5 -> 1 \
4 -> 0.5 \
3 -> 0\
2 -> 0 \
1 -> 0 

In [1277]:
new_ml = ml.copy()

new_ml.rating[new_ml.rating < 3] = 0
new_ml.rating[new_ml.rating == 4] = 0.5
new_ml.rating[new_ml.rating ==5] = 1
new_ml.rating[new_ml.rating ==3] = 0 

Формируем матрицы, которые подаются на вход LambdaRank_movie

In [1278]:
P_input = svd.P
Q_input = svd.Q
b_u = svd.U[:,1]
b_i = svd.I[1,:]
mu = svd.mu[1,1]

Обучаем LambdaRank_movie

In [1279]:
lambdarank = LambdaRank_movie(lr=0.005, sigma=2, num_epochs=4, seed=1,reg=0)

df = lambdarank.fit(P_input, Q_input, b_u, b_i, mu, new_ml)

100%|██████████| 943/943 [00:24<00:00, 39.06it/s]


### Сравниваем качество моделей по метрике NDCG
### Рекомендация обычно представляется в виде списка, блока, т.д. Поэтому помимо точности алгоритма, необходимо учитывать еще и порядок. Например, рекомендация в Youtube должна учитывать порядок, поскольку полезность модели будет тем выше, чем меньше пользователю придется листать до нужного контента.
### На наш взгляд, метрика NDCG удовлетворяет этим свойствам

In [1282]:
for n in [3,5,10]:
    print("--------")
    print(f"NDCG@{n} lambdarank",np.mean(lambdarank.compute_metric_for_all_queries(dct_val,15,n,NDCG)))
    print(f"NDCG@{n} svd",np.mean(svd.compute_metric_for_all_queries(dct_val,15,n, NDCG)))

--------
NDCG@3 lambdarank 0.6563426513694647
NDCG@3 svd 0.6562748048082077
--------
NDCG@5 lambdarank 0.6752036050568428
NDCG@5 svd 0.6718754990370885
--------
NDCG@10 lambdarank 0.7622794998195926
NDCG@10 svd 0.7612044572515383


## В данном примере LambdaRank лишь незначительно лучше SVDRank, поскольку признаки, используемые в алгоритме LambdaRank и SVDRank тождественны. Кроме того, также как и в первом алгоритме во втором используется линейная комбинация.

## Однако, качество все же выше во всех трех экспериментах, что объясняется настройкой именно на метрику NDCG